# Analysis and Visualization of E3SM Data using UXarray

E3SM Tutorial Workshop 2024

05/07/2024

Authors: [Tom Vo](https://github.com/tomvothecoder) and [Stephen Po-Chedley](https://github.com/pochedls)


## Overview

This exercise notebook will walkthrough and example end-to-end analysis workflow of E3SM
native format data using UXarray and xCDAT. It explores the capabilities of UXarray and xCDAT
at a high-level, including grid analysis and computational operations. Towards the end
there will brief coverage of parallelizing Xarray-based operations using Dask.

1. Open E3SM Data with Grid Files using UXarray
2. Visualize Grid Topology
3. Face Area Calculations
4. Subset an Unstructured Grid
5. Regrid Unstructured to Unstructured Grid

## Resources

- [UXarray Documentation](https://uxarray.readthedocs.io/en/stable/)


## Setup


In [ ]:
import glob

import numpy as np
import xarray as xr
import xcdat as xc
import uxarray as ux

# The data directory containing the NetCDF files.
# TODO: Update to perlmutter directory
data_dir = "/p/user_pub/work/E3SM/1_0/1950-Control-21yrContHiVol-HR/0_25deg_atm_18-6km_ocean/atmos/native/model-output/mon/ens1/v1/"
# The absolute paths to each NetCDF file in the data directory.
data_paths = glob.glob(data_dir + "*.nc")

# The path to the grid file.
grid_path = "/p/user_pub/e3sm/grids_maps/grids/ne120.g"

## I/O and Computations with UXarray

UXarray offers support for loading and representing unstructured grids by providing Xarray-like functionality paired with new routines that are specifically written for operating on unstructured grids.

Source: https://uxarray.readthedocs.io/en/latest/examples/001-working-with-unstructured-grids.html#


### Exercise 1. Open E3SM Dataset with Grid Files using UXarray

When working with Unstructured Grids, the grid definition and data variables are often stored as separate files. This means that there are multiple separate files that need to be read and linked together to represent the entire dataset.

A `ux.Dataset` object is an `xarray.Dataset-like`, multi-dimensional, in memory, array database. Inherits from `xarray.Dataset` and has its own unstructured grid-aware dataset operators and attributes through the uxgrid accessor.

Source: https://uxarray.readthedocs.io/en/latest/getting-started/overview.html


#### 💻 Your turn:

Use `ux.open_mfdataset()` to open the grid file and the NetCDF files as a `ux.Dataset` object.

Hint: Use `grid_path` and `data_paths` as function arguments.


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
uxds = ux.open_mfdataset(grid_path, data_paths[0:1])

### Exercise 2: Visualize the Grid Topology

In this exercise, we will visualize the topology of an unstructured grid (i.e., the
elements that make up a grid).

Exercises are copied from https://uxarray.readthedocs.io/en/latest/examples/006-plot-api-topology.html.


#### 💻 Your turn:

Extract the grid topology from the `uxds` and plot it.

Hint: Use the `.uxgrid` attribute and call `.plot()`


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
grid = uxds.uxgrid
grid.plot(title="Default Grid Plot Method", height=350, width=700)

### Exercise 3: Face Area Calculations

This section covers the different area calculation options provided by `uxarray`.
Note, this is a only subset of the available options.

Exercises are copied from https://uxarray.readthedocs.io/en/latest/examples/003-area-calc.html


#### 💻 Your turn:

Calculate the total face area for the grid.all

Hint: Use `.calculate_total_face_area()`


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
t4_area = grid.calculate_total_face_area()
t4_area

Calculate the total face area using the Quadratre Rule and Order of 4.

Order:

```
   1 to 10              for gaussian
   1, 4, 8, 10 and 12   for triangular
```


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
t1_area = grid.calculate_total_face_area(quadrature_rule="triangular", order=1)

Access the individual face areas using `Grid.face_areas`, then calculate the area using `Grid.compute_face_areas`.


In [ ]:
# Your code here. When ready, click on the three dots below for the solution.

In [ ]:
grid.face_areas

In [ ]:
all_face_areas, all_face_jacobians = grid.compute_face_areas(
    quadrature_rule="gaussian", order=4
)
g4_area = all_face_areas.sum()
g4_area

Now we compare the values with actual know value and report error for each of the three cases above.


In [ ]:
actual_area = 4 * np.pi
diff_t4_area = np.abs(t4_area - actual_area)
diff_t1_area = np.abs(t1_area - actual_area)
diff_g4_area = np.abs(g4_area - actual_area)

diff_t1_area, diff_t4_area, diff_g4_area

## Interoperability with xCDAT

Since `ux.UxDataset` and `ux.UxDataArray` extend the `xr.Dataset` and `xr.DataArray` classes,
_most_ xCDAT APIs are interoperable with UXarray objects.

- The exception is xCDAT's [spatial averager](https://xcdat.readthedocs.io/en/latest/generated/xarray.Dataset.spatial.average.html), which requires data on rectilinear grids. The data must first be remapped from unstructured to rectilinear grid using another tool like `nco`.
- There are plans to support unstructured to structured regridding in UXarray in the future.

Resources:

- [xCDAT Documentation Homepage](https://xcdat.readthedocs.io/en/stable/)
- [xCDAT API Reference Guide](https://xcdat.readthedocs.io/en/stable/api.html)

Feel free to jump over to the `xcdat_practicum_notebook.ipynb` to work with `nco` and `xcdat`.
